In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import warnings
import tensorflow as tf
import seaborn as sns
import sklearn
import random
import math
import time
import os

from scipy.stats import kurtosis, skew
from numpy.fft import fft


from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from scipy import optimize
from tensorflow.keras import optimizers


from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
from methods import *

%matplotlib inline
warnings.filterwarnings('ignore')

SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)

In [4]:
DEFAULT_SAMPLING_RATE = 100  # Fréquence d'échantillonnage en Hz

def plot_signal_pronostia(df, signal_name, unit=None):
    #     train = df
    plt.figure(figsize=(13,5))
    if unit:
        plt.plot('RUL_norm', signal_name,
                data=df[df['Unit']==unit])
    else:
        for i in df['Unit'].unique():
            # if (i % 10 == 0):  # only ploting every 10th unit_nr
            plt.plot('RUL_norm', signal_name, data=df[df['Unit']==i])
            
    plt.xlim(2560, 0)  # reverse the x-axis so RUL counts down to zero
    plt.xticks(np.arange(0, 2560, 250))
    plt.ylabel(signal_name)
    plt.xlabel('Remaining Use fulLife')
    #plt.savefig(signal_name+'.jpeg')
    plt.show()

# Chargement des données
def load_vibration_data(data_folder, idx=None):
    """Charge les fichiers de vibration et calcule le temps en secondes pour chaque échantillon."""
    vibration_files = [f for f in os.listdir(data_folder) if f.startswith('acc_') and f.endswith('.csv')]
    if not vibration_files:
        print("No vibration files found in the specified folder.")
        return pd.DataFrame()  # Retourne un DataFrame vide si aucun fichier n'est trouvé
    
    vibration_data = []

    for file in vibration_files:
        file_path = os.path.join(data_folder, file)
        df = pd.read_csv(file_path, names=['Hour', 'Minute', 'Second', 'Microsecond', 'Horizontal_Accel', 'Vertical_Accel'])
        
        # Extraction de l'identifiant du roulement
        bearing_id = file.split('_')[1].split('.')[0]
        df['bearing_id'] = idx + bearing_id
        
        # Calcul du temps en secondes pour chaque échantillon
        df['time_seconds'] = df['Hour'] * 3600 + df['Minute'] * 60 + df['Second'] + df['Microsecond'] * 1e-6
        vibration_data.append(df)

    # Concaténation de toutes les données de vibration en un DataFrame unique
    result = pd.concat(vibration_data, ignore_index=True)
    print(f"Loaded data shape: {result.shape}")
    print(result.head())
    return result

# Calcul du temps total et du RUL normalisé
def calculate_total_time(vibration_data):
    """Calcule le temps total de l'expérience pour chaque roulement."""
    return vibration_data.groupby('bearing_id')['time_seconds'].max()

def calculate_normalized_rul(vibration_data, total_time):
    """Calcule le RUL normalisé pour chaque échantillon."""
    vibration_data['RUL_norm'] = vibration_data.apply(
        lambda row: 1 - (total_time[row['bearing_id']] - row['time_seconds']) / total_time[row['bearing_id']],
        axis=1
    )
    return vibration_data

# Division en fenêtres
def split_into_windows(data, window_size, sampling_rate=DEFAULT_SAMPLING_RATE):
    """Divise les données en fenêtres de taille définie."""
    samples_per_window = int(window_size * sampling_rate)
    num_windows = len(data) // samples_per_window
    print(f"Data size: {len(data)}, Samples per window: {samples_per_window}, Number of windows: {num_windows}")
    if num_windows == 0:
        print("Warning: Not enough data to form a window.")
    windows = [data[i * samples_per_window:(i + 1) * samples_per_window] for i in range(num_windows)]
    return windows

# Extraction des caractéristiques temporelles
# def extract_temporal_features(signal):
#     """Extrait les caractéristiques temporelles."""
#     features = {
#         'mean': signal.mean(),
#         'std': signal.std(),
#         'peak_to_peak': signal.max() - signal.min(),
#         'rms': np.sqrt(np.mean(signal**2)),
#         'skewness': skew(signal),
#         'kurtosis': kurtosis(signal),
#     }
#     return features

# # Extraction des caractéristiques fréquentielles
# def extract_frequency_features(signal, sampling_rate=DEFAULT_SAMPLING_RATE):
#     """Extrait les caractéristiques fréquentielles."""
#     N = len(signal)
#     freqs = fft(signal)
#     freqs = np.abs(freqs[:N // 2])
#     freq_bins = np.fft.fftfreq(N, d=1 / sampling_rate)[:N // 2]
    
#     features = {
#         'max_amplitude': np.max(freqs),
#         'mean_freq': np.mean(freqs),
#         'rms_freq': np.sqrt(np.mean(freqs**2)),
#     }
#     return features

# Extraction de toutes les caractéristiques
def extract_features_per_window(data, window_size, sampling_rate=DEFAULT_SAMPLING_RATE):
    """Extrait les caractéristiques de chaque fenêtre de données."""
    features_list = []

    for bearing_id, group in data.groupby('bearing_id'):
        print(f"Processing bearing: {bearing_id}, Data size: {len(group)}")
        horizontal_signal = group['Horizontal_Accel'].values
        vertical_signal = group['Vertical_Accel'].values
        
        horizontal_windows = split_into_windows(horizontal_signal, window_size, sampling_rate)
        vertical_windows = split_into_windows(vertical_signal, window_size, sampling_rate)
        
        total_duration = group['time_seconds'].max()
        
        for i, (h_window, v_window) in enumerate(zip(horizontal_windows, vertical_windows)):
            temporal_features_h = extract_temporal_features(h_window)
            temporal_features_v = extract_temporal_features(v_window)
            frequency_features_h = extract_frequency_features(h_window)
            frequency_features_v = extract_frequency_features(v_window)
            
            features = {f"{k}_h": v for k, v in temporal_features_h.items()}
            features.update({f"{k}_v": v for k, v in temporal_features_v.items()})
            features.update({f"{k}_freq_h": v for k, v in frequency_features_h.items()})
            features.update({f"{k}_freq_v": v for k, v in frequency_features_v.items()})
            
            # Calculer le RUL normalisé pour cette fenêtre
            start_time_window = i * window_size  # Temps de début de la fenêtre en secondes
            RUL = max(0, total_duration - start_time_window)  # RUL décroît avec le temps
            RUL_norm = RUL / total_duration if total_duration > 0 else 0  # RUL normalisé
            
            # Ajouter des informations contextuelles
            features['bearing_id'] = bearing_id
            features['window_index'] = i
            features['RUL_norm'] = RUL_norm  # Ajout de RUL normalisé
            
            features_list.append(features)

    return pd.DataFrame(features_list)

# Pipeline principal
def main_pipeline(data_folder, idx, window_size=10, sampling_rate=DEFAULT_SAMPLING_RATE):
    """Pipeline complet pour traiter les données."""
    # Charger les données
    vibration_data = load_vibration_data(data_folder, idx)
    if vibration_data.empty:
        print("No data loaded. Check your data folder.")
        return pd.DataFrame()
    
    # Calculer le temps total et le RUL normalisé
    total_time = calculate_total_time(vibration_data)
    vibration_data = calculate_normalized_rul(vibration_data, total_time)
    print(f"Data after RUL calculation: {vibration_data.shape}")
    
    # Extraire les caractéristiques
    features_df = extract_features_per_window(vibration_data, window_size, sampling_rate)
    print(f"Extracted features shape: {features_df.shape}")
    
    return features_df

def main(data_folder):
    """Fonction principale pour charger les données, calculer le temps total et le RUL normalisé."""
    # Charger les données de vibration
    vibration_data = load_vibration_data(data_folder)
    
    # Calculer le temps total de l'expérience pour chaque roulement
    total_time = calculate_total_time(vibration_data)
    
    # Calculer le RUL normalisé
    vibration_data_with_rul = calculate_normalized_rul(vibration_data, total_time)
    
    return vibration_data_with_rul


In [40]:
import numpy as np
from scipy.fft import fft
from scipy.stats import kurtosis, skew

def extract_frequency_features(signal, sampling_rate=25600):
    """Extrait les caractéristiques fréquentielles d'un signal donné."""
    # Convertir le signal en tableau NumPy
    signal = np.asarray(signal)

    # Vérifier si le signal est vide après conversion
    if signal.size == 0:
        raise ValueError("Signal vide : impossible de calculer des caractéristiques fréquentielles.")

    # Appliquer la FFT
    N = len(signal)
    freqs = fft(signal)
    freqs = np.abs(freqs[:N // 2])  # On garde seulement les composantes positives
    freq_bins = np.fft.fftfreq(N, d=1/sampling_rate)[:N // 2]
    
    # Calcul des caractéristiques fréquentielles
    max_amplitude = np.max(freqs)
    mean_freq = np.mean(freqs)
    rms_freq = np.sqrt(np.mean(freqs**2))
    variance_freq = np.var(freqs)
    std_freq = np.std(freqs)
    kurtosis_freq = kurtosis(freqs)
    skewness_freq = skew(freqs)
    peak_freq = freq_bins[np.argmax(freqs)]
    form_factor_freq = rms_freq / mean_freq if mean_freq != 0 else 0
    crest_factor_freq = max_amplitude / rms_freq if rms_freq != 0 else 0

    features = {
        'max_amplitude': max_amplitude,
        'mean_freq': mean_freq,
        'rms_freq': rms_freq,
        'variance_freq': variance_freq,
        'std_freq': std_freq,
        'kurtosis_freq': kurtosis_freq,
        'skewness_freq': skewness_freq,
        'peak_freq': peak_freq,
        'form_factor_freq': form_factor_freq,
        'crest_factor_freq': crest_factor_freq
    }
    return features

# Caractéristiques temporelles
def extract_temporal_features(signal):
    """Extrait les caractéristiques temporelles : moyenne, écart type, RMS, skewness, etc."""
    features = {
        'mean': signal.mean(),
        'std': signal.std(),
        'peak_to_peak': signal.max() - signal.min(),
        'rms': np.sqrt(np.mean(signal**2)),
        'mean_abs': np.mean(np.abs(signal)),
        'max_abs': np.max(np.abs(signal)),
        'skewness': skew(signal),
        'kurtosis': kurtosis(signal),
        'form_factor': np.sqrt(np.mean(signal**2)) / np.mean(np.abs(signal)),
        'crest_factor': signal.max() / np.sqrt(np.mean(signal**2)),
        'impulse_factor': signal.max() / np.mean(np.abs(signal)),
        'margin_factor': signal.max() / (np.mean(np.sqrt(np.abs(signal))) ** 2)
    }
    return features

In [41]:
# Utilisation de la fonction principale

idx = '1_2'
folder = 'PRONOSTIA/Learning_set/'
bearing = 'Bearing' + idx
base_dir = os.path.join(folder, bearing)


data_folder = base_dir
vibration_data_with_rul = main_pipeline(data_folder, idx=idx,window_size= 10)
# print(vibration_data_with_rul[['bearing_id', 'time_seconds', 'RUL_norm']].head())

Loaded data shape: (2229760, 8)
   Hour  Minute  Second  Microsecond  Horizontal_Accel  Vertical_Accel  \
0     8      47       5     196910.0             0.050          -0.253   
1     8      47       5     196950.0             0.165          -0.140   
2     8      47       5     196990.0             0.125           0.542   
3     8      47       5     197030.0             0.157          -0.261   
4     8      47       5     197070.0             0.421           0.081   

  bearing_id  time_seconds  
0   1_200001   31625.19691  
1   1_200001   31625.19695  
2   1_200001   31625.19699  
3   1_200001   31625.19703  
4   1_200001   31625.19707  
Data after RUL calculation: (2229760, 9)
Processing bearing: 1_200001, Data size: 2560
Data size: 2560, Samples per window: 1000, Number of windows: 2
Data size: 2560, Samples per window: 1000, Number of windows: 2
Processing bearing: 1_200002, Data size: 2560
Data size: 2560, Samples per window: 1000, Number of windows: 2
Data size: 2560, Samples

In [1]:
import pickle
nom_fichier = "df_training.pkl"

with open(nom_fichier, "rb") as fichier:
    list_df_training = pickle.load(fichier)

print("Liste rechargée :", list_df_training)

Liste rechargée : [        mean_h     std_h  peak_to_peak_h     rms_h  mean_abs_h  max_abs_h  \
0       0.1944  0.535807           1.687  0.569983      0.5122      0.885   
1      -0.0810  0.386241           1.206  0.394643      0.3262      0.649   
2       0.1537  0.533026           1.759  0.554743      0.4331      1.149   
3       0.0416  0.283630           1.025  0.286665      0.2376      0.560   
4      -0.0145  0.321055           1.003  0.321382      0.2815      0.538   
...        ...       ...             ...       ...         ...        ...   
717563  0.2778  2.079979           6.791  2.098448      1.7696      4.145   
717564  0.5264  2.475479           7.577  2.530828      2.2780      3.837   
717565  0.1330  1.956694           6.338  1.961209      1.6472      3.872   
717566  1.4288  4.310002          12.424  4.540659      4.2766      6.533   
717567 -1.4301  3.158040          10.005  3.466757      3.1295      5.507   

        skewness_h  kurtosis_h  form_factor_h  crest_fac

(222976, 225)